In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [ ]:
import sys

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [ ]:
data_date = "2018_06_27_12_39_39"
data_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/" + data_date

In [ ]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
params.print()

In [ ]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 342
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

# Read the data

In [ ]:
scalingFactor = 2 / 65535
offset = -32768

In [ ]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

In [ ]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

In [ ]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [ ]:
plt.figure()
plt.plot( A_reshaped[0, :] )
title = "first acquisition " + data_date
plt.title(title)
plt.grid()
plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

**Set the shifted flag properly**

In [ ]:
rampDownFirst = 1

## Spectrum

### No window

In [ ]:
samplesPerDownRamp = int(samplesPerRamp/2)

In [ ]:
fft_A = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [ ]:
fft_abs = 20 * np.log10( np.abs( fft_A ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs[::100, 0:1000] )
title = "no window " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Hamming

In [ ]:
hamming = np.hamming(samplesPerDownRamp)

In [ ]:
fft_A_hamming = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hamming, axis=1)

In [ ]:
fft_abs_hamming = 20 * np.log10( np.abs( fft_A_hamming ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_hamming[::100, 0:1000], cmap='gray' )
title = "hamming " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Hanning

In [ ]:
hanning = np.hanning(samplesPerDownRamp)

In [ ]:
fft_A_hanning = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hanning, axis=1)

In [ ]:
fft_abs_hanning = 20 * np.log10( np.abs( fft_A_hanning ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_hanning[::100, 0:1000], cmap='gray' )
title = "hanning " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Blackman (peu utilisée, cf lff)

In [ ]:
blackman = np.blackman(samplesPerDownRamp)

In [ ]:
fft_A_blackman = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*blackman, axis=1)

In [ ]:
fft_abs_blackman = 20 * np.log10( np.abs( fft_A_blackman ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_blackman[::100, 0:1000], cmap='gray' )
title = "blackman " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Kaiser

In [ ]:
# beta Window shape
# 0    Rectangular
# 5    Similar to a Hamming
# 6    Similar to a Hanning
# 8.6  Similar to a Blackman
# kaiser = np.kaiser(samplesPerDownRamp)

In [ ]:
fft_A_kaiser = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*kaiser, axis=1)

In [ ]:
fft_abs_kaiser = 20 * np.log10( np.abs( fft_A_kaiser ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_kaiser[::100, 0:1000], cmap='gray' )
plt.title("kaiser " + data_date)

# Build RD

In [ ]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
withHanning = 1
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

In [ ]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
if withHanning:
    np.save(data_dir + '/RD_files_{}_{}_rampDown_hanning'.format(firstFile, lastFile), RDc) 
else:
    np.save(data_dir + '/RD_files_{}_{}_rampDown'.format(firstFile, lastFile), RDc)